<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/7%20-%20Topic%20modelling/topic_modelling_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic modelling con Scikit-Learn
Latent Dirichlet allocation (LDA) è un modello statistico che ci permette di associare ogni documento del nostro corpus di testo a degli argomenti (**topic**) quantificando quanto il documento è inerente a tale argomento.
Ad esempio, prendiamo la frase: "Il calciatore e la velina si sono spostati a Parigi", l'analisi LDA potrebbe stabilire che la frase riguarda al 80% gossip e al 20% sport. I topic del LDA non contengono il nome specifico dell'argomento (come in questo caso sport o gossip) ma le parole chiave del topic, quindi sta a noi utilizzando queste risalire al topic.
<br><br>
In questo notebook cerchermo di identificare gli argomenti di oltre un milione di titoli di giornale presenti all'interno del dataset utilizzando LDA con scikit-learn.

## Otteniamo il dataset
Il dataset contenente un milione di titoli di giornale è presente su Kaggle [a questo indirizzo](https://www.kaggle.com/therohk/million-headlines), per scaricare l'ultima versione aggiornata devi registrarti su Kaggle. Se non vuoi registrarti al momento puoi scaricare una versione più vecchia del dataset da [questo indirizzo ](https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv). Se utilizzi Google Colab o comunque hai wget installato sul tuo computer esegui pure la cella di codice qui sotto per scaricare il dataset.

In [1]:
!wget https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv

--2019-04-17 09:00:04--  https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54096356 (52M) [text/plain]
Saving to: ‘abcnews-date-text.csv’

abcnews-date-text.c 100%[===================>]  51.59M  59.4MB/s    in 0.9s    

2019-04-17 09:00:17 (59.4 MB/s) - ‘abcnews-date-text.csv’ saved [54096356/54096356]



Il dataset è un file CSV, carichiamo all'interno di un DataFrame con pandas.

In [2]:
import pandas as pd

headlines_df = pd.read_csv("abcnews-date-text.csv")
headlines_df.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


Utilizziamo il metodo *.info()* per ottenere qualche informazione sul DafraFrame.

In [3]:
headlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1082168 entries, 0 to 1082167
Data columns (total 2 columns):
publish_date     1082168 non-null int64
headline_text    1082168 non-null object
dtypes: int64(1), object(1)
memory usage: 16.5+ MB


Come vedi il dataset ha la bellezza di 1082168 di titoli di giornale, l'ideale sarebbe utilizzarli tutti per il nostro modello, ma questo richiederebbe molto tempo per la creazione del modello, quindi selezioniamone casulamente un 20% utilizzando il metodo *.sample()*.

In [4]:
# impostiamo il random_state a 0
# per ottenere gli stessi valori

headlines_df = headlines_df.sample(frac=.2, random_state=0)
headlines_df.shape

(216434, 2)

Abbiamo ridotto il DataFrame a sole (si fa per dire) 216434 righe. Estraiamo l'array numpy.

In [5]:
headlines = headlines_df["headline_text"].values
headlines[:3]

array(['labor attacks nationals mp over milk link',
       'coraki public school hip hop video', 'dairy record'], dtype=object)

## Modello Bag of Words
Adesso dobbiamo codificare il testo in un formato comprensibile per un computer, utilizziamo il bag of words con la classe *CountVectorizer* di sklearn, limitiamo il numero di parole a 5000 e rimuoviamo le stop words.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer(max_features=5000, stop_words="english")
X = bow.fit_transform(headlines)
X.shape

(216434, 5000)

Adesso possiamo eseguire la LDA utilizzando la classe *LatentDirichletAllocation* di sklearn, trattandosi di un problema di **apprendimento non supervisionato** non conosciamo a priori il numero di topic da cercare, definiamolo arbitrariamente a 10 utilizzando il parametro *n_components*.

In [7]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

lda = LDA(n_components=10, max_iter=10, verbose=True)
lda.fit_transform(X)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


## Esplorazione dei topic
All'interno dell'attributo *.components_* troviamo la probabilità di appartenenza di ogni parola ad un determianto topic.

In [11]:
topic = lda.components_[0]
lda.components_[0].shape

(5000,)

Ad esempio, stampiamo la probabilità di appartenenza al primo topic delle prime 10 parole del dizionario. Possiamo ottenere il nostro dizionario utilizzando il metodo .get_feature_names() del bag of words.

In [13]:
print("PAROLA\t\tPROBABILITA'")
for i in range(0,10):
  print("%s\t\t%.4f" % (bow.get_feature_names()[i], topic[i]))

PAROLA		PROBABILITA'
10		1.2008
100		0.1000
1000		0.1000
10000		0.1000
100k		0.1000
100m		54.8004
10m		0.1000
11		0.7890
12		7.1969
13		0.1000


Il metodo *.argsort()* di un array numpy ci permette di ottenere gli indici dell'array ordinato, usiamolo per ottenere gli indici delle 10 parole più popolari del primo topic.

In [14]:
top_words = topic.argsort()[-10:]
top_words

array([ 495, 3730,  358, 4927, 1708, 3953, 2705, 1149, 4963, 4916])

Usiamo questa informazione per stampare le prime 10 parole più popolari del topic.

In [15]:
for i in top_words:
  print(bow.get_feature_names()[i])

big
rise
australian
wins
final
set
market
cup
world
win


Adesso mettiamo tutto insieme per stampare le 10 parole più popolari di tutti i topic.

In [0]:
n_words = 10

for index, topic in enumerate(lda.components_):
  print("\nTOPIC %d - %d parole più popolari" % (index+1, n_words))
  print([bow.get_feature_names()[i] for i in topic.argsort()[-n_words:]])


TOPIC 1 - 15 parole più popolari
['high', 'centre', 'action', 'anti', 'police', 'threat', 'warning', 'pay', 'port', 'open', 'deal', 'opposition', 'house', 'health', 'school']

TOPIC 2 - 15 parole più popolari
['says', 'report', 'risk', 'tasmanian', 'workers', 'climate', 'work', 'continue', 'debate', 'continues', 'final', 'study', 'missing', 'change', 'search']

TOPIC 3 - 15 parole più popolari
['residents', 'urges', 'rural', 'help', 'flood', 'guilty', 'public', 'calls', 'sa', 'farmers', 'urged', 'qld', 'home', 'govt', 'water']

TOPIC 4 - 15 parole più popolari
['mp', 'ban', 'government', 'industry', 'australian', 'iraq', 'talks', 'pm', 'claims', 'election', 'labor', 'says', 'minister', 'govt', 'australia']

TOPIC 5 - 15 parole più popolari
['bomb', 'pakistan', 'plane', 'gas', 'kills', 'china', 'bail', 'people', 'car', 'attack', 'crash', 'injured', 'woman', 'dead', 'killed']

TOPIC 6 - 15 parole più popolari
['race', 'long', 'nrl', 'media', 'return', 'man', 'face', 'told', 'drug', 'fac

Riesci a riconoscere il significato di ogni topic ?

## Modello TF-IDF
Ricreiamo il modello, questa volta codificando le frasi utilizzando un modello TF-IDF, possiamo farlo con la classe *TfidfVectorizer* di sklearn.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000, stop_words="english")
X = tfidf.fit_transform(headlines)
X.shape

(216434, 5000)

E ricreiamo il modello utilizzando questo nuovo dataset.

In [17]:
lda = LDA(n_components=10, max_iter=10, verbose=True)
data = lda.fit_transform(X)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


Esploriamo i topic generati con lo stesso metodo visto sopra.

In [18]:
n_words = 10

for index, topic in enumerate(lda.components_):
  print("\nTOPIC %d - %d parole più popolari" % (index+1, n_words))
  print([bow.get_feature_names()[i] for i in topic.argsort()[-n_words:]])


TOPIC 1 - 10 parole più popolari
['obama', 'new', 'carbon', 'cattle', 'abbott', 'tax', 'pm', 'climate', 'says', 'change']

TOPIC 2 - 10 parole più popolari
['boost', 'greens', 'plan', 'funds', 'urged', 'government', 'rail', 'council', 'funding', 'govt']

TOPIC 3 - 10 parole più popolari
['study', 'care', 'safety', 'concerns', 'cancer', 'warning', 'death', 'new', 'indigenous', 'health']

TOPIC 4 - 10 parole più popolari
['dead', 'woman', 'missing', 'killed', 'dies', 'charged', 'car', 'crash', 'police', 'man']

TOPIC 5 - 10 parole più popolari
['sport', 'new', 'rejects', 'farm', 'national', 'weather', 'business', 'news', 'rural', 'abc']

TOPIC 6 - 10 parole più popolari
['cuts', 'budget', 'job', 'drought', 'toll', 'nrn', 'record', 'gold', 'rain', 'coast']

TOPIC 7 - 10 parole più popolari
['smith', 'team', 'michael', 'season', 'australia', 'cup', 'tour', 'world', 'asylum', 'interview']

TOPIC 8 - 10 parole più popolari
['cup', 'blues', 'test', 'share', 'open', 'tigers', 'day', 'final', 

## Visualizzare il modello
[pyLDAvis](https://github.com/bmabey/pyLDAvis) è un fantastico modulo python che ci permette di esplorare i topic generati da un modello LDA in maniera visuale, installiamolo usando pip.

In [19]:
!pip install pyldavis

    100% |████████████████████████████████| 1.6MB 13.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


Utilizziamo per creare la visualizzazione per un modello sklearn, passando come argomenti il modello stesso, il dataset codificato e l'oggetto che abbiamo usato per la codifica. Per poter visualizzare il grafico dobbiamo proiettare i dati in uno spazio bi-dimensionale, all'interno del parametro mds possiamo definire la tecnica per farlo, utilizziamo il [t-distributed stochastic neighbor embedding](https://it.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding).

In [25]:
import pyLDAvis.sklearn

lda_viz = pyLDAvis.sklearn.prepare(lda, X, tfidf, mds='tsne')
pyLDAvis.display(lda_viz)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
